# Part 1: Matching Datasets

## Project Introduction

This project focuses on analysing the relationship between altitude and population of global cities... [section to be completed after finishing part 2]

The primary dataset used is the simplemaps dataset [1]. The simplemaps dataset does not contain elevation, the elevation is extracted from the secondary dataset, geonames [2] . The two datasets to not share the same unique identifier for each city - it is necessary to join based on the city name and country. This doesn't provide a unique match for each city, so the population and latitude/longitude are used to identify matches. Part 1 of this project aims to merge the 2 datasets.

Note: There is nuance to the definition of a "city", but for simplicity, in this project a "city" refers to a populated area. Each entry in each dataset is considered a "city".


## Data Sources

### Dataset 1: simplemaps

The simplemaps dataset was used for the following reasons:
- Recentness: the dataset was last updated on 11/5/25 (Same time as the commencement of this project)
- Simplicity: population, city and city name are all given with one row per entry
- Size: there are approximately 48,000 cities in the dataset which is both comprehensive, and not overwhelmingly large 
- Source: the dataset is built from reputable sources (NGIA, US Geological Survey, US Census Bureau, and NASA)

Upon critical evaluation, the sources are all tied to entities associated with the United States, which may give a biased or inaccurate representation of international cities. However, given that this dataset measures population and location, one can assume there is very minimal bias introduced from the geopolitical climate of the United States.

Upon regarding the visualisation of the dataset by location (as given by simplemaps, [1]):

![Figure 1: Map of every city in the simplemaps dataset, provided in [1]](plots_part1/simple%20maps%20map.png)

Without delving into the full complexities of international population density, several observations can be drawn from Figure 1.

Firstly there seems to be bias, with a very strong density of cities in the United States, the United Kingdom and South Eastern Europe. Whilst these are densely populated areas, it does seem as though these cities may be over-represented, perhaps due to ease of access of information. This is acknowledged, and will be taken into account in any conclusions drawn.

Secondly, one can identify that there is data on North Korea (as well as other regions, with a complex geopolitical situation). It is hard to verify data from North Korea, due to the complexities of the region. It is possible there is data here that may be misleading or inaccurate. Due to North Korea's smaller representation in the dataset, it is likely that bias will be negligible, however it is acknowledged.

There is sparsity of towns in certain regions (like North Africa, Siberia, and central Australia) however this is very consistent with these areas being largely uninhabitable due to deserts other conditions. 

The columns of interest have information on: city name, country, population, latitude, longitude and ID. These are all self explanatory.


### Dataset 2: geonames

The simplemaps dataset, does not include information on altitude/elevation. It's consequently necessary to introduce another dataset to join on. This is the geonames dataset (cities 1000) [2]. The dataset chosen is the cities1000.txt dataset, to include every city with a population over 1000.

The geonames dataset is sourced from multiple international agencies, and thus contains a broader range of data, making it less consistent than the simplemaps dataset.

The columns of interest have information on: city name, country, population, latitude, longitude, ID and DEM.

The geonames dataset includes altitude and DEM (Digital Elevation Model, approximate distance above sea level according to satellites). Given that most cities don't have an altitude value, the DEM is used.

The following visualisation is created using geonames_vis.ipynb (in \Part 1\Python).

![Figure 2: Map of every city in the geonames dataset](plots_part1/geonames%20map.png)

A slightly different means of visualising cities was chosen to the visualisation provided by simplemaps. Notably, an increased size and transparency of mark was chosen to represent cities due to the increased number of cities (approximately 150,000) in the dataset. The larger dataset makes visualising each city by a point more challenging, consequentially more transparent yet larger marks are used to give a better impression of density of coverage.

Figure 2 shows a broader coverage of cities than Figure 1, with greater density of coverage in places like India, Europe and Thailand. However the presentation style can be misleading to appear to have a greater frequency of cities. This map again shows bias, with disproportionately high density in Europe and the United States. This perhaps reflects the bias of the availability of geographical data, and matches general trends in Figure 1.

A summary table of the datasets is given below:

| Feature          | simplemaps                     | geonames |
|------------------|--------------------------------|---------------------------------------------|
| Source           | simplemaps.com                 | geonames.org                                |
| Last Updated     | 11 May 2025                    | Varies by region; generally 2023–2024       |
| Licence/Use      | Free for educational use       | Free for educational use                    |
| Cities        | 48,056 cities           | 154,694 cities      |
| Columns of interest     | City name, country, lat/lng, population, ID | City name, country, lat/lng, population, ID, DEM |
| Elevation Data   | No                          | DEM (Digital Elevation Model)            |
| Data Consistency | High                           | Mixed (multiple international sources)      |

## Sample Datasets and Similarity Metrics
Going forward it is necessary to join the two datasets to extract the DEM from geonames, to use in simplemaps. The problem is there is no obvious way to match towns. Whilst it may seem intuitive to match on city name and country it proves problematic. This problem gives rise to this, part 1, of the project.

To investigate this, one can use an SQL query (as in '/Part 1/cities database.sqbpro'). The first "section" reports how many repeats of each city name and country combination are in the geonames dataset:

| City              | Country | Count |
|-------------------|---------|-------|
| Emiliano Zapata   | MX      | 27    |
| Benito Juarez     | MX      | 23    |
| Clinton           | US      | 23    |
| Guadalupe Victoria| MX      | 21    |
| Lazaro Cardenas   | MX      | 21    |
| San Miguel        | PH      | 21    |
| Springfield       | US      | 20    |
| Franklin          | US      | 19    |
| San Vicente       | PH      | 19    |
| Greenville        | US      | 18    |


This shows it would be impossible to match cities based on solely name and country. Fortunately, both datasets share population and longitude/latitude. It is consequentially possible to use those measures to match cities. However, it is not as simple as matching exact numbers as there is some disparity in population and location between datasets. One can assume that two cities are the same if they have "similar" population and location, this notion leads to the following investigation. The goal is to "match" cities to themselves between datasets.

A "match" in this project refers to the combination of entries from both datasets, of a city which can be uniquely identified in both datasets. 

To roughly outline this using notation:

Denote the simplemaps and geonames datasets each as (simplemaps) and (geonames). (City, Country_code) is used to refer to an entry in either dataset.

If (London, UK) $\in$ (simplemaps) can be uniquely identified with (London, UK) $\in$ (geonames) (and vice versa) then (London, UK) $\times$ (London, UK) $\in$ (simplemaps $\times$ geonames) can be considered a "match".

On the other hand, if there are $n$ (Springfield, US)s $\in$ (simplemaps), and $m$ (Springfield, US)s $\in$ (geonames). Label these as (Springfield, US)$_i$ $\in$ (simplemaps) for $i = 1,...,n$ and (Springfield, US)$_j$ $\in$ (geonames) for $j = 1,...,m$. There are $m\dot n$ "potential matches" of (Springfield, US)$_i$ $\times$ (Springfield, US)$_j$ $\in$ (simplemaps $\times$ geonames) $\forall i,j$. The goal is then to deduce which potential matches are actual matches. This can be done using data available in the datasets (population and location) or manual validation.

For simplicity, matches are assumed to be injective (each city in one dataset will match to at most one other city in another dataset).


### Sample Cities: Known Matches

In order to determine how cities should be matched, two sample sets are chosen.

The first sample set can be used to identify how population and coordinates vary in known matched cities. This set should:
    - Have cities that are likely to be unique in their own country (so they can be verified as matches) 
    - Represent a broad range of cities, by both population and location (to investigate all kinds of disparities)

This set of cities was sampled from an LLM. The LLM was prompted to give a sample of random cities (which are unique by name in their own country) that covered all continents (excluding Antarctica and including the Middle East) and varying sizes. 

An LLM was used:
    - to reduce human bias
    - for the knowledge of place names that are likely to be without repeats - a human (myself) would be unlikely to have such a broad knowledge set

For ethics, the output was verified by a human (myself) and was concluded to be a good representation of the dataset. The uniqueness of each location is inspected further.

The LLM chose the following cities:

| Region         | Sampled Cities                                                                 |
|----------------|---------------------------------------------------------------------------------|
| North America  | New York (US), Boulder (US), Banff (CA), Chicago (US), Vancouver (CA)          |
| South America  | Lima (PE), Valparaíso (CL), Bariloche (AR), Medellín (CO), Quito (EC)          |
| Europe         | London (GB), Pisa (IT), Inverness (GB), Munich (DE), Zagreb (HR), Porto (PT)   |
| Africa         | Nairobi (KE), Casablanca (MA), Banjul (GM), Lusaka (ZM), Durban (ZA)           |
| Asia (exc. Middle East)           | Tokyo (JP), Chiang Mai (TH), Leh (IN), Hanoi (VN), Tbilisi (GE)                |
| Oceania        | Sydney (AU), Dunedin (NZ), Alice Springs (AU), Wellington (NZ), Cairns (AU)    |
| Middle East    | Beirut (LB), Amman (JO), Muscat (OM), Riyadh (SA), Doha (QA)                   |

The consideration of the Middle East as a continent over Antarctica is a fault in the LLM's reasoning, however, this error is constructive to the goals of the project. Antarctica is too unpopulated to sample cities from, and breaking up Asia (the largest and most populated continent) gives better coverage of an otherwise broad area. 

Investigation reveals that there are two counts of a city names "Boulder" the United States in the geonames dataset. Using coordinates, one is in Colorado (which matches with the Boulder in the simplemaps dataset). The second Boulder is in Montana, which is obviously a different city. This is still included in this sample set (for an example of a false match).

### Sample Cities: Potentially Ambiguous Cities


A second sample set should be chosen to see what disparities between the two datasets look like. An LLM was again prompted to give city names which may prove ambiguous and returned the following list:

- Springfield (US)
- Kingston (JM)
- Victoria (CA)
- San Jose (CR)
- St. Louis (US)
- Sao Luis (BR)

The only cities in this list that have multiple entries in either dataset are Springfield in the US (11 counts in simplemaps, 20 in geonames) and San Jose (1 count in simplemaps, 2 counts in geonames). Since the other cities have verifiable matches between datasets, they are included in the first sample set. These cities may introduce a slight bias towards cities in the Americas. This bias is likely to be negligible as the selection of cities is fairly arbitrary, and positively impacts the diversity of the sample set.

This highlights the shortcomings of using an LLM for sampling. This selection is fairly arbitrary and is used for preliminary investigation, so the shortcomings are unlikely to have a negative impact further downstream. 

There are 2 $(2 \times 1)$ potential matches of San Joses between the two datasets and 220 $(11 \times 20)$ for Springfields. The Springfields are analysed for preliminary analysis of how disparities manifest (so the San Joses are dropped from the dataset). Since matches are assumed to be injective, there is a maximum of 11 matches between the Springfields (assuming each Springfield in the simplemaps dataset is in the geonames dataset) and a minimum of 0.


### Measurements of Similarity: Distance

It is necessary to outline how to measure the similarity in population and location between a city in both datasets. 

When matching distance, it may be sensical to use the co-ordinates to directly distance (perhaps Manhattan distance, or Euclidean). The Earth, however, is spherical - measuring distance with latitude/longitude would bias distances between cities that are further from the equator.

It is possible to measure the distance between two co-ordinate points. This is achieved through the Haversine formula. Which is used to measure distance between two points modelling the Earth as a sphere. The formula is widely used in geospatial analysis and uses the Earth's radius (approximately 6371km) and trigonometric functions. The exact details are omitted for brevity, but the formula is given in geonames_vis.ipynb. 

### Measurements of Similarity: Population

Measuring similarity in population may sound more simple. However areas with larger populations are likely to have different magnitudes of population difference. An example is Alice Springs in Australia (verified match) which has populations of 23,726 and 25,186 in the simplemaps and geonames datasets respectively (1,460 difference). On the other hand, a much larger city like Hanoi (verified match) has populations of 8,587,100 and 8,053,663 (533,437 difference) which is much larger.

It would be impossible to determine a static boundary on population difference to determine if there is a match, without introducing bias against smaller cities. Instead, one might be inclined to use ratios of population. For example for Alice Springs and Hanoi, the simplemaps population differs from the geonames population by 6.2% and 6.6% respectively, and shows that the populations don't differ too much.

To create a statistic that is invariant in both scale, and order of population, the following is used: $$\text{ALPR}(x_1,x_2) = \text{exp}(\left | \log\left(\frac{x_1}{x_2} \right) \right|)$$

Where $x_1$ = population according to simplemaps and $x_2$ = population according to geonames (these are both values > 0).

For $\alpha >0$ it can be shown that:
$$
\text{ALPR}(x, \alpha x) = \exp\left( \left| \log\left( \frac{x}{\alpha x} \right) \right| \right) = \exp\left( |\log(\alpha^{-1})| \right) =
\begin{cases}
\alpha, & \text{if } \alpha \geq 1 \\
\frac{1}{\alpha}, & \text{if } 0<\alpha < 1
\end{cases}


$$
The converse, $ALPR(x,\alpha x)$ can be shown to give the same result, showing the symmetry of the function. These results show the ALPR is an effective, scale invariant measure of population ratio.

The exponents and logarithms are all calculated in base 10, but are arbitrary as the cancel out in the derivation. The outer exponent isn't entirely necessary, but is used to make the output on the same scale as the ratio itself, making the output more interpretable.

This is henceforth referred to the "Absolute Log Population Ratio" (ALPR). Cities with a population of 0 are considered to be incomplete data, and are consequentially not used, ensuring no dividing by 0 errors. It can be shown that the log population ratio is invariant under labelling (through splitting the logarithm, taking out a factor of -1, and then merging the logarithms).

For example, the ALPR between Alice Springs is 1.061, and Hanoi, 1.065. This score indicates high similarity, showing that these populations are on the same scale. Contrarily, a mismatched pair, like between Hanoi in the simplemaps dataset and Alice Springs in the geonames dataset gives an ALPR of 5.832. This larger score indicates that the two populations are on different scales.

It should be noted that the difference in populations of Tokyo is 28,051,724. (This is likely due to the one dataset measuring the central Tokyo area, compared to the Greater Tokyo area). This can be considered an outlier, but is likely to occur with other major cities. This problem is noted and taken into account further downstream.



## Visualising Sample Matches

To summarise the previous sections, there are two sample sets of cities. The first contains a broad range of cities which are easily matches between datasets (and one instance of Boulder - Montana, included as a reference point). The second contains Springfields in the United States.

As a side note: the ambiguity of the place name Springfield, in the United States is famously played on within "The Simpsons". There is a running joke that the Simpsons live in Springfield but the writers never state which one, but humorously tease it.

A dataset is constructed to contain every potential match of cities, grouped by city name and country code. The first part of the dataset is designed to have one match per city (with the exception of Boulder). The second part of the dataset contains each of the 220 potential matches of Springfield, US.

The initial sample dataset is created in section 2 in database.sqbpro.

This dataset is imported to Excel for visualisation. This file is titled "city sample check.xlsx" under "\part 1". 

The following scatter-plots plot distance between cities (through log Haversine distance) and difference in population (through ALPR) with the aim of identifying a boundary between correct matches and false matches. Known matches are shown in orange, and false matches are identified in blue.The log of the Haversine Distance is used for better visualisation.

The known matches are shown in Figure 3:

![Figure 3: Scatterplot of known matches (and Boulder)](plots_part1/samples_matches.png)

There is clear behaviour of known matches, as they all seem to cluster with low difference in population scale and low distance between them. The inclusion of the incorrectly matched Boulder shows that there is a clear difference in behaviour.

The Springfield $\times$ Springfield matches were validated by manually and evaluated by the distance between them. It was found that there were 10 potential matches which were less than 5km apart (considered matches). One potential match was found to have two Springfields approximately 25km away. Upon investigation, these two Springfields refer to two different towns, in the suburbs of Philadelphia, in Pennsylvania, US. Consequentially these two are not a match. Figure 4 is used to further investigate differences between correct and incorrect matches.

![Figure 4: Scatterplot of potential matches for Springfield, US](plots_part1/Springfield_matches.png)

Again there is a clear cluster of known matches. From Figure 4 one might be inclined to draw a boundary with Haversine distance around $10^{1} = 10\text{km}$ and ALPR of $\approx 3$. That is to say that any potential match with populations that are different by a factor less than 2 within 10km of each other, is considered a match. This is not a one size fits all approach. These boundaries are tested on the whole dataset.

## Matching Every City

The previous process is extended to the entirety of both datasets. Similar to before, a dataset is created which includes every potential match for each unique (City, Country_code). The length of potential matches for each unique (City, Country_code) ($m \times n$) is counted and assigned to each dataset entry. This will be referred to as the length of a set of potential matches. Potential matches with a length of 1 indicate that (City, Country_code) is unique in both datasets. The Haversine distance and ALPR is also calculated for this dataset. 

This dataset can be visualised, similarly to before, using python (under 'Part 1/python/cities_check_vis.py').

This code, when run (with Log axes, and boundaries ascertained by previous sections) results in Figure 5. Figure 5 displays the relationship between distances between potential matches, and difference in population. Every potential match set with a length of 1 (implying that this (City, Country_name) is unique, i.e. likely match) is given a green colour. Sets of potential matches with length greater than 1 are mapped to a colour map with: $2 \mapsto$ yellow and (the maximum set length in the dataset) $\mapsto$ red. Other values can be linearly interpolated to a shade of orange. This colour encoding is used to show how ambiguous a potential match is. 

![Figure 5: Every Potential Matched City between simplemaps and geonames datasets](plots_part1/scatterplot_matching_cities_init.png)

Figure 5 shows a very clear distinction between 2 clusters, as well as a concentration of points where the distance = $10^{-6} = 0.000001$ - the offset factor in the logarithm for visualisation, where the distance between cities is 0 because they are identical. The log scale makes these points seem dissimilar to those seemingly centred around distance of $10^{-3} = 0.001$. However they can be considered the same cluster.

Upon visual inspection, a more accurate distance boundary can be drawn with at distance value $ 30\text{km} \approx 10^{1.48}$. It is clear that using ALPR does seem to be as effective in creating a meaningful split in the data, and the boundary is not very successful. Drawing a more distinct boundary (at 30km) for distance and removing the ALPR boundary, results in the following plot (Figure 6). The choice of 30km as a distance boundary is not chosen rigourously, merely based on visual inspection. This could be done according to various sophisticated techniques, but 30km visually performs well, so is selected. 

The exclusion of a boundary for population difference (as calculated by the ALPR), allows for mismatches in city specifications (like Tokyo and the Greater Tokyo area) to be included within the dataset. This resolves the issue of mismatched cities with varying definitions.

![Figure 6: Every Potential Matched City between simplemaps and geonames datasets (new boundaries)](plots_part1/scatterplot_matching_cities.png)

### Visualisation Evaluation

In Figure 6, the two clusters have a range of populations and have a clear split at the adjusted boundary for distance, indicating that this may be a very clear indication of correct and incorrect matches. 

The cluster that lies below the distance boundary has mostly green points, indicating that this set is mostly comprised of potential matches with a unique (City,Country_name). Upon closer inspection, it is possible to identify points that fall in the yellow to red spectrum. This is to be expected as every set of potential matches should have some correct matches - as seen with Springfield, US. It is possible that there are some potential matches that fall in cluster that are false matches. This is because they have not all been manually verified - however it is unlikely there is an overwhelming amount.

The cluster that lies above the boundary is comprised mostly of non-green points, indicating that these points are mostly ambiguous - however this is not sufficient to determine these are false matches. The green points in this cluster indicate either:
- There are (City, Country_code) entries that are unique in both datasets, but do not refer to the same place (for example: the green points that are $10^{4} = 10000\text{km}$ away)
- There are some correct matches that are falsely identified as incorrect, according to the distance boundary (potentially the cities that fall just short of the boundary)

Both cases show that this is a non-perfect method. This cluster has a broader spread of population, with greater difference reflecting the ambiguity of these potential matches.

If this project were to approach with more rigour, one could:
 - Apply clustering or classification techniques
 - Impose Gaussian distributions on the clusters (which seems like it would be successful) and cluster according to GMMs
 - Compare quantiles of a Gaussian distribution and the clusters to validate use of GMMs
 - Use a more rigorous training/testing construction, this project only loosely follows these notions

A dataset of "True" matches is created by including every match with cities that are within 30km of each other. It is necessary to validate the uniqueness of matches in this dataset before proceeding to part 2 of this project. 

## Final Validation

An SQL query (as in section 5 in database.sqbpro) reveals that there are 194 matches with duplicate IDs in the dataset of 31344 matches. Assuming that every other city is correctly matched, this gives (an assumed) precision of 99.38% which can be thought of as very successful.

Investigating the false positive matches shows a few places of note that slipped through the cracks: 

The match associated with the Springfields in Pennsylvania. This is consistent with the prior investigation, which showed that these two cities were suburbs of Philadelphia. These two cities are unusually close and show just how common the city name "Springfield" is in the United States.

Kansas City, US also proved problematic. This is due to the fact that this city is interestingly divided into two districts, as it lies directly on a state boundary. The Western district is considered part of Kansas and the Eastern, part of Missouri. Thus the existence of two "Kansas Cities" right next to each other.

It is acknowledged that these cities could be aggregated or de-duplicated, however these cities form less than 1% of the dataset. It is justifiable to remove them as any further investigation could introduce unnecessary complexity. 


## Conclusion

From Figure 6, it is clear a boundary of 30km works well. The resulting dataset (with duplicate filtering) is used in part 2, which aims to explore the relationship between altitude and population of cities. This dataset can be used with trust in the simplemaps dataset, with added information provided by the geonames dataset. The matching between areas helps to create a dataset which can be used effectively in part 2.

## References

## Note

If you are reading this, part 2 is still not complete, and this report may undergo tweaking and refinement to support further conclusions in part 2. 

As of June 2025, I intend on focussing my efforts on another project, to maintain/build a broad skill set.

### [1] simplemaps dataset 
- Available at: https://simplemaps.com/data/world-cities
- Free to use for educational purposes (see site for more details)
- Accessed May 2025

### [2] geonames dataset
- Available at: https://download.geonames.org/export/dump/ (cities 1000 dataset)
- Free to use for educational purposes
- Accessed May 2025
